<img src="./img/img-3.png"><br> 

## データセット
<div style="border:1px solid #000; padding:10px;">
量子化学計算を行う前の準備として、分子構造のコンフォーマー探索を行い、安定な構造骨格を得る必要があります。本手法では、RDKitライブラリによる力場計算（UFFまたはMMFF[1]）を用いてコンフォーマーを探索し、最も安定な構造を基に、Gaussian用の入力ファイル（.gjf）、psi4用の.xyzファイル、あるいはmol形式のファイルとして出力します。


[1] Tosco, P., Stiefl, N. & Landrum, G. *Bringing the MMFF force field to the RDKit: implementation and validation.* J Cheminform **6**, 37 (2014). https://doi.org/10.1186/s13321-014-0037-3  

---

## RDKitで扱えるUFFおよびMMFFの分子力場について

### **Universal Force Field (UFF)**  
Universal Force Field（UFF）はコロラド州立大学にて開発された汎用分子力場であり、アクチノイドを含む周期表全元素に対してパラメータが定義されている点に特長があります。元素については横断的な適用性を有するため、広範な分子系に対して構造最適化やコンフォーマー探索を可能とします。一方で、特に金属錯体や無機化合物においては、経験的パラメータの精度や実験データとの整合性に課題があり、定量的な信頼性には限界があります。

### **Merck Molecular Force Field (MMFF)**    
Merck Molecular Force Field（MMFF）は、Merck Research Laboratories によって開発された分子力場であり、従来の MM3 力場の設計思想を継承・発展させたものです。MMFF は有機分子全般に対して高い再現性と精度を発揮するように設計されていることに特長があります。一方で、下記の「MMFFのコンフォーマー生成における留意点 」にあるような、分子構造によっては計算に困難を伴うことがあります。

### MMFFのコンフォーマー生成における留意点  
以下に示す構造的特徴を有する分子は、MMFFに基づく構造最適化およびコンフォーマー生成において困難を伴うことがあります。

- **長鎖アルキル基**  
  十分な柔軟性を持つ長鎖アルキル基は、多数の回転異性体を生み出しうるため、コンフォーマー空間が極めて広大となり、探索アルゴリズムの収束に時間を要することがあります。
  
- **複雑な縮合環系**  
  多環芳香族や縮合複素環を有する分子は、構造の剛直性が高く、コンフォーマー空間が限られるため、探索効率が低下する傾向があります。

- **嵩高い置換基の存在**  
  イソプロピル基やトリフルオロメチル基といった立体的にかさばる置換基は、コンフォーマー生成アルゴリズムにおいて不自然な衝突やコンフォーマーの収束失敗を引き起こす要因となります。

- **電荷を有する官能基**  
  正電荷を有する四級アンモニウム（[N+]）、負電荷を有するボロネートアニオン（[B-]）などの存在は、静電ポテンシャル場の形成に大きく影響し、構造最適化の収束を妨げる可能性があります。

- **ボロン (B) を含む構造**  
  ボロンは多様な配位数と結合様式を示し、MMFFとの整合性が乏しい場合があります。特に、ボロンに窒素や芳香環が直接結合している構造では、立体的拘束が大きく、初期構造生成に失敗することがあります。

- **ボロン-窒素 (B–N) 結合環**  
  一部の分子においては、B–N結合を含む芳香族様の環構造が確認されており、電子的特性や結合の特殊性が力場のパラメータ化と整合しない可能性がある。
  
---

### 教材への接続
google colabにおけるオンラインの場合にこのラインを実行します。（<font color="red">Google colabに接続しない場合には不要</font>）

In [ ]:
!pip install rdkit
!git clone https://github.com/ARIM-Academy/Advanced_Tutorial_2.git
%cd Advanced_Tutorial_2

### ライブラリのインポート
RDkitでコンフォーマーサーチを行うためのpythonのライブラリをimport文でロードします。

In [1]:
#汎用ライブラリ
import glob, os
import pandas as pd
import traceback

#コンフォーマー用
from rdkit import Chem
from rdkit.Chem import AllChem

# オリジナルモジュール
from util import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 初期設定
confs = 1000        # コンフォーマーの発生数
rms = 2.0           # 類似した構造を排除する枝刈りのレベルの設定
pickup = 1          # コンフォーマーの選択数
forcefield = "uff"  # UFF (Universal Force Field)  または MMFF（Merck Molecular Force Field、メルク分子力場）

#出力フォルダ
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# エラーログフォルダ
error_dir = "error_log"
os.makedirs(error_dir, exist_ok=True)

# エラーとなった骨格を記録するリスト
failed_molecules = []

## 1.データセットの読み込み
### SMILESファイルの読み込み
化合物のデータセットを読み込み、各化合物のID情報とSMILES表記を取得し、これから行う分子のコンフォーマー生成や力場計算の準備をします。まず、pandasライブラリのread_csv()関数は、SMILESが記載されているCSVファイルを読み込んでpandasのDataFrame形式に変換する関数です。ここでは、[data]フォルダ内に保存されている.csvファイルをDataFrameとして読み込み、その結果をdfという変数に格納します。

In [3]:
# --- 分子情報の読み込み ---
df = pd.read_csv('data/smiles_10.csv')
labels = df["ID"]
smiles_list = df["SMILES"]

In [4]:
smiles_list [0]
mol = Chem.MolFromSmiles(smiles_list [0])

## 2.コンフォーマー生成
RDKit を使って、分子に対して複数のコンフォーマー（立体配座）を生成し、それらを最適化してエネルギーの低い順に並べるユーザー関数を定義します。最もエネルギーの低い構造を基準にして、**相対エネルギー**として返します。返り値は (相対エネルギー, コンフォーマーID)のリスト構造です。

In [6]:
# コンフォーマー生成
def opt_conformers(mol, forcefield):
    try:
        cids = AllChem.EmbedMultipleConfs(mol, 
                                        numConfs = confs, 
                                        pruneRmsThresh = rms, 
                                        randomSeed = 1234
                                        )
        
        # コンフォーマ生成に失敗した場合
        if len(cids) == 0:
            return None
            
        energies = []
        
        if forcefield == 'uff':
            for cid in cids:
                ff = AllChem.UFFGetMoleculeForceField(mol, confId=cid)
                ff.Minimize()
                energies.append((ff.CalcEnergy(), cid))

        elif forcefield == 'mmff':
            props = AllChem.MMFFGetMoleculeProperties(mol)
            if props is None:  # MMFFプロパティが取得できない場合
                return None
                
            for cid in cids:
                ff = AllChem.MMFFGetMoleculeForceField(mol, props, confId=cid)
                ff.Minimize()
                energies.append((ff.CalcEnergy(), cid))
        
        energies.sort()
        
        return [(e - energies[0][0], cid) for e, cid in energies]
    except Exception as e:
        return None

### 【解説】
`EmbedMultipleConfs()` は分子に複数の 3D 構造（コンフォーマー）を埋め込むメソッド（関数）です。
```python
cids = AllChem.EmbedMultipleConfs(mol, 
                                  numConfs = confs, 
                                  pruneRmsThresh = rms, 
                                  randomSeed = 1234)
```

- 
- `numConfs=confs`：生成するコンフォーマーの個数。
- `pruneRmsThresh=rms`：同じような構造（RMSD が近い）を間引く閾値。
- `randomSeed=1234`：再現性のための乱数シード。
- `cids`：生成されたコンフォーマーの ID（インデックス）のリスト。

---

### forcefield が `'uff'` のとき

```python
ff = AllChem.UFFGetMoleculeForceField(mol, confId=cid)
ff.Minimize()
energies.append((ff.CalcEnergy(), cid))
```

- UFF（Universal Force Field）を用いて最適化。
- `Minimize()` でエネルギー最小化。
- `CalcEnergy()` で最小化後のエネルギーを取得。
- `(エネルギー, コンフォーマーID)` のタプルとしてリストに追加。

### forcefield が `'mmff'` のとき

```python
props = AllChem.MMFFGetMoleculeProperties(mol)
```

- MMFF（Merck Molecular Force Field）ではまず `props`（分子の性質情報）を取得する必要があります。

```python
ff = AllChem.MMFFGetMoleculeForceField(mol, props, confId=cid)
ff.Minimize()
energies.append((ff.CalcEnergy(), cid))
```

- `props` を使ってMMFF用の力場を生成。
- `Minimize()` → `CalcEnergy()` → 追加、は UFF と同様。


In [7]:
# エラーログを記録する関数
def log_error(smiles, label, error_msg):
    with open(os.path.join(error_dir, "error_log.txt"), "a") as f:
        f.write(f"ID: {label}, SMILES: {smiles}\n")
        f.write(f"Error: {error_msg}\n")
        f.write("-" * 50 + "\n")

## 3.実行およびファイル出力
SMILES 文字列リストから分子を作成 → 水素を付加 → 複数コンフォーマーを生成・最適化させます。
その後、最安定となった構造について.mol, .xyz, .gjf ファイルとして保存を行います。

In [ ]:
%%time

# ファイル出力ループ
print("処理を開始します...")

for i, (label, smiles) in enumerate(zip(labels, smiles_list)):
    try:
        # 水素付加
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            print(f"警告: 無効なSMILES ({smiles}) - スキップします")
            failed_molecules.append((label, smiles, "無効なSMILES形式"))
            log_error(smiles, label, "無効なSMILES形式")
            continue
            
        mol = Chem.AddHs(mol)

        # コンフォーマーの生成と最適化
        results = opt_conformers(mol, forcefield)
        
        if results is None or len(results) == 0:
            print(f"警告: コンフォーマー生成に失敗 ({smiles}) - スキップします")
            failed_molecules.append((label, smiles, "コンフォーマー生成失敗"))
            log_error(smiles, label, "コンフォーマー生成失敗")
            continue
            
        selected = results[:pickup]

        # ファイル出力
        for j, (rel_energy, cid) in enumerate(selected):
            print(f"{i+1}/{len(smiles_list)} - {smiles} 処理中...")
            base = f"{label}_{forcefield}_{j}"
            
            # molファイル保存
            molfile = os.path.join(output_dir, base + ".mol")
            Chem.MolToMolFile(mol, molfile, confId=cid)
            
            # xyzファイル保存
            xyzfile = os.path.join(output_dir, base + ".xyz")
            write_xyz_input(mol, cid, xyzfile)
            
            # gjfファイル保存
            gjf_file = os.path.join(output_dir, base + ".gjf")
            write_gaussian_input(mol, cid, gjf_file)

    except Exception as e:
        error_msg = traceback.format_exc()
        print(f"エラー発生 ({smiles}): {str(e)}")
        failed_molecules.append((label, smiles, str(e)))
        log_error(smiles, label, error_msg)
        continue

# 処理結果の表示
print(f"処理完了: {output_dir} フォルダに .mol, .xyz, .gjf が保存されました。")
print(f"エラーログ: {error_dir}/error_log.txt に保存されました。")

処理を開始します...
1/10 - C1(C2=NN(C3=CC=CC4=C3C=CC=C4)N=C2)=CC=CC=C1 処理中...
2/10 - O=C(C1=CC=CC=C1)C2=NN(C3=CC=CC=C3)N=C2C4=CC=CC=C4 処理中...
3/10 - C1(C2=NN(C3=NC=CC=C3)N=C2)=CC=CC=C1 処理中...
4/10 - FC(C=C1)=CC=C1C2=NN(C3=CC=C(F)C=C3)N=C2 処理中...


### エラーとなった分子のサマリーファイルの作成

In [ ]:
# エラーとなった分子の要約
if failed_molecules:
    print(f"\n処理に失敗した分子数: {len(failed_molecules)}/{len(smiles_list)}")
    
    # エラーサマリーをCSVで保存
    error_summary = pd.DataFrame(failed_molecules, columns=["ID", "SMILES", "Error"])
    error_summary.to_csv(os.path.join(error_dir, "failed_molecules.csv"), index=False)
    print(f"失敗した分子のリスト: {error_dir}/failed_molecules.csv に保存されました。")
else:
    print("\nすべての分子が正常に処理されました。")

## 4.最安定構造の可視化
util.pyの補助関数 show_3D_from_xyz() を使ってpy3Dmolから、最安定構造のコンフォーマーを可視化します。

In [ ]:
def show_all_xyz_from_output(folder='output', pattern='*.xyz'):
    """
    指定フォルダ内の「0番目構造（最安定構造）」のxyzファイルをすべて表示する。
    
    Args:
        folder: xyzファイルが格納されているフォルダ名（デフォルト: 'output'）
        pattern: 検索パターン（デフォルト: '*_0.xyz'）

    Returns:
        None（Notebook上で順に表示される）
    """
    file_paths = sorted(glob.glob(os.path.join(folder, pattern)))

    if not file_paths:
        print("該当ファイルが見つかりませんでした。")
        return

    for xyz_path in file_paths:
        show_3D_from_xyz(xyz_path)


In [ ]:
show_all_xyz_from_output()